In [1]:
from __future__ import print_function
import os
import neat

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
from explaneat.visualization import visualize

from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler


In [2]:
random.seed(4242)
nGenerations = 1

In [3]:
def one_hot_encode(vals):
    width = max(vals)
    newVals = []
    for val in vals:
        blank = [0. for _ in range(width + 1)]
        blank[val] = 1.
        newVals.append(blank)
    return np.asarray(newVals)


In [4]:
iris = datasets.load_iris()
xs_raw = iris.data[:, :2]  # we only take the first two features.
scaler = StandardScaler()
scaler.fit(xs_raw)
xs = scaler.transform(xs_raw)
ys = iris.target
ys_onehot = one_hot_encode(ys)

In [5]:
def eval_genomes(genomes, config):
    loss = nn.CrossEntropyLoss()
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds), torch.tensor(ys)))

In [6]:
config_path = "./config-iris-fast"

In [7]:
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

# Create the population, which is the top-level object for a NEAT run.
p = BackpropPopulation(config, xs, ys, criterion=nn.CrossEntropyLoss())

# Add a stdout reporter to show progress in the terminal.
p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)
p.add_reporter(neat.Checkpointer(5))
bpReporter = backprop.BackpropReporter(True)
p.add_reporter(bpReporter)

In [8]:
# Run for up to nGenerations generations.
winner = p.run(eval_genomes, nGenerations)


 ****** Running generation 0 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.9079, grad_fn=<DivBackward0>)
Population's average fitness: 0.89944 stdev: 0.13705
Best fitness: 1.10148 - size: (3, 6) - species 1 - id 1
ending generation %s
Average adjusted fitness: 0.142
Mean genetic distance 2.851, standard deviation 1.525
Population of 6 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0     1      1.1    0.262     0
     2    0     3      1.0    0.131     0
     3    0     2      0.8    0.035     0
Total extinctions: 0
Generation time: 18.176 sec


In [9]:
p.reporters.reporters[2].save_checkpoint(p.config, p.population, p.species, str(p.generation) + "-final")  

# Display the winning genome.
print('\nBest genome:\n{!s}'.format(winner))

# Show output of the most fit genome against training data.
print('\nOutput:')
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
# for xi, xo in zip(xor_inputs, xor_outputs):
#     output = winner_net.activate(xi)
#     print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))

results = []
for xi, xo in zip(xs, ys):
    output = winner_net.activate(xi)
    print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))
    results.append([xi[0], xi[1], output])

df = pd.DataFrame(results)
df.to_csv('results.csv')

node_names = {-1:'A', -2: 'B', 0:'A XOR B'}
visualize.draw_net(config, winner, True)
visualize.plot_stats(stats, ylog=False, view=False)
visualize.plot_species(stats, view=False)

Saving checkpoint to neat-checkpoint-1-final

Best genome:
Key: 1
Fitness: 1.1014796495437622
Nodes:
	0 DefaultNodeGene(key=0, bias=0.6730118989944458, response=1.0, activation=sigmoid, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=0.9788631796836853, response=1.0, activation=sigmoid, aggregation=sum)
	2 DefaultNodeGene(key=2, bias=-0.2758592963218689, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=1.0910375118255615, enabled=True)
	DefaultConnectionGene(key=(-2, 1), weight=-0.23812013864517212, enabled=True)
	DefaultConnectionGene(key=(-2, 2), weight=-0.4458010494709015, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=-1.8317101001739502, enabled=True)
	DefaultConnectionGene(key=(-1, 1), weight=0.073113352060318, enabled=True)
	DefaultConnectionGene(key=(-1, 2), weight=0.7178376317024231, enabled=True)

Output:
input array([-0.90068117,  1.01900435]), expected output 0, got [0.9999999651723205, 0.9661646945308718, 

FileNotFoundError: [Errno 2] No such file or directory: 'xdg-open': 'xdg-open'

In [10]:
# p = run(config_path, 0)

g = p.best_genome

config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

net = NeatNet(g, config) 

winnerNet = neat.nn.FeedForwardNetwork.create(g, config)

ancestry = p.reporters.reporters[3].trace_ancestry_of_species(g.key, p.reproduction.ancestors) 

print('have ancestry')

ancestors = {
    k: v['genome'] for k, v in p.reporters.reporters[3].ancestry.items()
}
print('have ancestors')
visualize.create_ancestry_video(p.config, g, ancestry, ancestors, p.reporters.reporters[1])
print('have finished video')

trying to cudafy
trying to cudafy
trying to cudafy
trying to cudafy
trying to cudafy
trying to cudafy
trying to cudafy
trying to cudafy
trying to cudafy
gen is 0
previous generation is [1]
skey is 1
have calculated the ancestry
have ancestry
have ancestors
Fitness for gen 0


Format: "png" not recognized. Use one of: canon cmap cmapx cmapx_np dot dot_json eps fig gv imap imap_np ismap json json0 mp pdf pic plain plain-ext png pov ps ps2 svg svgz tk vml vmlz xdot xdot1.2 xdot1.4 xdot_json


0
{1: 1.1014796495437622}


CalledProcessError: Command '['dot', '-Tpng', '-O', 'tmp/ancestor0000000000']' returned non-zero exit status 1.

In [11]:
net.params

[Parameter containing:
 tensor([-1.8317], device='cuda:0', requires_grad=True), Parameter containing:
 tensor([0.0731], device='cuda:0', requires_grad=True), Parameter containing:
 tensor([0.7178], device='cuda:0', requires_grad=True), Parameter containing:
 tensor([1.0910], device='cuda:0', requires_grad=True), Parameter containing:
 tensor([-0.2381], device='cuda:0', requires_grad=True), Parameter containing:
 tensor([-0.4458], device='cuda:0', requires_grad=True), Parameter containing:
 tensor([0.6730], device='cuda:0', requires_grad=True), Parameter containing:
 tensor([0.9789], device='cuda:0', requires_grad=True), Parameter containing:
 tensor([-0.2759], device='cuda:0', requires_grad=True)]